### Part 3.2.1 - Compare the different models trained on part 3.2 for the model 1

The purpose of the notebook is to select the best model among the 16 models trained with the multi-input keras approach.
The final model selected will be then compared to the rest of the models trained.

To select the best model we used the following guidelines:

* 1) The model with the lowest hamming loss & zero one loss
* 2) The model with the lowest test score and the highest test accuracy values
* 3) The model with the most accurate predictions among the 17 labels. It is of high importance the best model to identify correctly the most of the genre tags. Models that cannot identify more than 2 genre tags will not be prefered.
* 4) Compare model predictions on movie never seen before.
* 5) Training-Validation metrics comparison.

Set the version data control parameter (to save the outputs of this notebook at their latest date)

In [ ]:
version_data_control="16072020"

#### Import the libraries

In [ ]:
import collections

try:
    collectionsAbc = collections.abc
except AttributeError:
    collectionsAbc = collections

import pandas as pd
import numpy as np
import os
import time
import random
import pickle
import json
import shutil
import unidecode

%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from datetime import datetime
from tabulate import tabulate
from packaging import version
from humanfriendly import format_timespan
from sklearn.metrics import confusion_matrix, classification_report, hamming_loss, zero_one_loss, f1_score, roc_auc_score

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
date_format='%Y-%m-%d %H-%M-%S'

#### Improt visualization libraries

In [ ]:
%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
import plotly.graph_objects as go

import pydot
import pydotplus
import graphviz

from IPython.display import SVG
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=False)
from pylab import rcParams

#### Import Tensorflow

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_docs as tfdocs #!pip install git+https://github.com/tensorflow/docs
import tensorflow_docs.plots as tfplots
import tensorflow_docs.modeling as tfmodel

from tensorflow.keras import layers, regularizers, models
from tensorflow.keras import models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import model_to_dot, plot_model
from tensorflow.keras.models import load_model, model_from_json

#### Import Tensorflow Hub

In [ ]:
import tensorflow_hub as hub
from tensorboard.plugins.hparams import api as hp

#### Import Keras

In [ ]:
import keras.backend as K
from tensorflow import keras

In [ ]:
print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

#### Import the data already tokenized and transformed from Part 3.1

* 80-20 split - Non-balanced data

In [ ]:
saved_version_data_control="13072020"
tokenization_history_folder="text_tokenization_padded_sequences"

with open(os.path.join(os.getcwd(), '80-20 split_non-balanced\\{0}_{1}\\words_tokenized_{1}.pkl'.format(tokenization_history_folder, saved_version_data_control)), 'rb') as handle:
    words_tokenized = pickle.load(handle)
    
words_tokenized

#### Import the saved tokenizers

In [ ]:
"""
IMport the tokenizers of each input, fitted on part 3.1
"""
with open(os.path.join(os.getcwd(), '80-20 split_non-balanced\\{0}_{1}\\actors_tokenizer_{2}_{1}.pkl'.format(tokenization_history_folder, saved_version_data_control, words_tokenized['actors_tokenized'])),'rb') as f:
    actors_tokenizer = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '80-20 split_non-balanced\\{0}_{1}\\plot_tokenizer_{2}_{1}.pkl'.format(tokenization_history_folder, saved_version_data_control, words_tokenized['plot_words_tokenized'])),'rb') as f:
    plot_tokenizer = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '80-20 split_non-balanced\\{0}_{1}\\features_tokenizer_{2}_{1}.pkl'.format(tokenization_history_folder, saved_version_data_control, words_tokenized['features_words_tokenized'])),'rb') as f:
    features_tokenizer = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '80-20 split_non-balanced\\{0}_{1}\\reviews_tokenizer_{2}_{1}.pkl'.format(tokenization_history_folder, saved_version_data_control, words_tokenized['reviews_words_tokenized'])),'rb') as f:
    reviews_tokenizer = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '80-20 split_non-balanced\\{0}_{1}\\title_tokenizer_{2}_{1}.pkl'.format(tokenization_history_folder, saved_version_data_control, words_tokenized['title_words_tokenized'])),'rb') as f:
    title_tokenizer = pickle.load(f)

try:
    assert len(actors_tokenizer.word_index)==words_tokenized['actors_tokenized']
    assert len(plot_tokenizer.word_index)==words_tokenized['plot_words_tokenized']
    assert len(features_tokenizer.word_index)==words_tokenized['features_words_tokenized']
    assert len(reviews_tokenizer.word_index)==words_tokenized['reviews_words_tokenized']
    assert len(title_tokenizer.word_index)==words_tokenized['title_words_tokenized']
except AssertionError:
    print("ERROR: The vocabulary length for some of the tokenizers, is not equal to 20000. Please verify their lengths by running the following: len(actors_tokenizer.word_index)")

print("Tokenizers are loaded successfully!")

In [ ]:
saved_version_data_control="13072020"
tokenization_history_folder="text_tokenization_padded_sequences"

X_train_seq_actors=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\{0}_{1}\\x_train_seq_actors_80-20_non-balanced_{2}_{1}.npy".format(tokenization_history_folder, saved_version_data_control, words_tokenized['actors_tokenized'])))
X_train_seq_plot=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\{0}_{1}\\x_train_seq_plot_80-20_non-balanced_{2}_{1}.npy".format(tokenization_history_folder, saved_version_data_control, words_tokenized['plot_words_tokenized'])))
X_train_seq_features=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\{0}_{1}\\x_train_seq_features_80-20_non-balanced_{2}_{1}.npy".format(tokenization_history_folder, saved_version_data_control, words_tokenized['features_words_tokenized'])))
X_train_seq_reviews=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\{0}_{1}\\x_train_seq_reviews_80-20_non-balanced_{2}_{1}.npy".format(tokenization_history_folder, saved_version_data_control, words_tokenized['reviews_words_tokenized'])))
X_train_seq_title=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\{0}_{1}\\x_train_seq_title_80-20_non-balanced_{2}_{1}.npy".format(tokenization_history_folder, saved_version_data_control, words_tokenized['title_words_tokenized'])))

print("X_train data inputs have been loaded!\n")

X_test_seq_actors=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\{0}_{1}\\x_test_seq_actors_80-20_non-balanced_{2}_{1}.npy".format(tokenization_history_folder, saved_version_data_control, words_tokenized['actors_tokenized'])))
X_test_seq_plot=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\{0}_{1}\\x_test_seq_plot_80-20_non-balanced_{2}_{1}.npy".format(tokenization_history_folder, saved_version_data_control, words_tokenized['plot_words_tokenized'])))
X_test_seq_features=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\{0}_{1}\\x_test_seq_features_80-20_non-balanced_{2}_{1}.npy".format(tokenization_history_folder, saved_version_data_control, words_tokenized['features_words_tokenized'])))
X_test_seq_reviews=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\{0}_{1}\\x_test_seq_reviews_80-20_non-balanced_{2}_{1}.npy".format(tokenization_history_folder, saved_version_data_control, words_tokenized['reviews_words_tokenized'])))
X_test_seq_title=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\{0}_{1}\\x_test_seq_title_80-20_non-balanced_{2}_{1}.npy".format(tokenization_history_folder, saved_version_data_control, words_tokenized['title_words_tokenized'])))

print("X_test data inputs have been loaded!\n")

y_train=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\{0}_{1}\\y_train_80-20_non-balanced_{1}.npy".format(tokenization_history_folder, saved_version_data_control)))
y_test=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\{0}_{1}\\y_test_80-20_non-balanced_{1}.npy".format(tokenization_history_folder, saved_version_data_control)))

print("y_train & y_test have been loaded!\n")

In [ ]:
# Import genres

with open(os.path.join(os.getcwd(), 'pickled_data_per_part\\genres_list_06032020.pkl'.format(tokenization_history_folder, saved_version_data_control)),'rb') as f:
    genres_list = pickle.load(f)
genres_list

In [ ]:
X_train_seq_actors.shape

In [ ]:
y_test.shape

Initialise some predefined values first:

* Set Optimization function
* Model loss
* Model metric

In [ ]:
# Initialize name variables

saved_model_name="multi_input_keras_model"
folder_path_model_saved="model_one\\adam_v2_models_20072020"
saved_df_scored_metric_name="df_metrics_multi_input_keras_model"

In [ ]:
neural_network_parameters={}
optimizer_parameters={}

#----------------------------------------------------------------------

# Model Compilation
neural_network_parameters['model_loss'] = tf.keras.losses.BinaryCrossentropy(from_logits=False, name='binary_crossentropy') #'binary_crossentropy'
neural_network_parameters['model_metric'] = [tfa.metrics.HammingLoss(mode="multilabel", name="hamming_loss"),
                                             tfa.metrics.F1Score(y_train.shape[-1], average="micro", name="f1_score_micro"), 
                                             tfa.metrics.F1Score(y_train.shape[-1], average=None, name="f1_score_none"),
                                             tfa.metrics.F1Score(y_train.shape[-1], average="macro", name="f1_score_macro")]

#----------------------------------------------------------------------
# Function 1 - Optimizers

# Optimizer: ADAM (Learning scheduler with Inverse Time Decay)

optimizer_parameters['lr_scheduler_decay_rate'] = 0.1
optimizer_parameters['staircase'] = False
optimizer_parameters['validation_split_ratio']=0.7

def optimizer_adam_v2(hparams):

    return keras.optimizers.Adam(tf.keras.optimizers.schedules.InverseTimeDecay(
        initial_learning_rate=hparams[HP_LEARNING_RATE],
        decay_steps=int(np.ceil((X_train_seq_actors.shape[0]*optimizer_parameters['validation_split_ratio'])//hparams[HP_HIDDEN_UNITS]))*hparams[HP_DECAY_STEPS_MULTIPLIER],
        decay_rate=optimizer_parameters['lr_scheduler_decay_rate'],
        staircase=optimizer_parameters['staircase']))
#---------------------------------------------------------------------------------------

# Optimizer: SDG (version 1)

optimizer_parameters['SGD_momentum'] = 0.2 #default 0.0
optimizer_parameters['SGD_nesterov'] = True #default False

def optimizer_sgd_v1(haparms, mode):

    if mode=="step decay":

        return keras.optimizers.SGD(lr=0.0, #Notice that we set the learning rate in the SGD class to 0 to clearly indicate that it is not used.
                                    momentum=0.9 #Use a large momentum. Using a larger momentum value will help the optimization algorithm to continue to make updates in the right direction when your learning rate shrinks to small values.
                                   )
    else:
        return keras.optimizers.SGD(lr=hparams[HP_LEARNING_RATE],
                              momentum=optimizer_parameters['SGD_momentum'],
                              nesterov=optimizer_parameters['SGD_nesterov'])

#---------------------------------------------------------------------------------------

# Optimizer: RMSprop (version 1)

optimizer_parameters['RMSprop_momentum'] = 0.5
optimizer_parameters['RMSprop_centered'] = True

def optimizer_rmsprop_v1(haparms):

    return keras.optimizers.RMSprop(lr=hparams[HP_LEARNING_RATE],
                                    momentum=optimizer_parameters['RMSprop_momentum'],
                                    centered=optimizer_parameters['RMSprop_centered'])

#----------------------------------------------------------------------

# Function 2
def hamming_loss(y_true, y_pred, mode='multilabel'):
    if mode not in ['multiclass', 'multilabel']:
        raise TypeError('mode must be: [multiclass, multilabel])')

    if mode == 'multiclass':
        nonzero = tf.cast(tf.math.count_nonzero(y_true * y_pred, axis=-1), tf.float32)
        print(nonzero)
        return 1.0 - nonzero

    else:
        nonzero = tf.cast(tf.math.count_nonzero(y_true - y_pred, axis=-1), tf.float32)
        return nonzero / y_true.shape[-1]

class HammingLoss(tfa.metrics.MeanMetricWrapper):
    def __init__(self, name='hamming_loss', dtype=None, mode='multilabel'):
        super(HammingLoss, self).__init__(hamming_loss, name, dtype=dtype, mode=mode)

In [ ]:
# Function 1

def import_trained_keras_model(method, decay_steps_mode, optimizer_name, hparams):
    """
    Load the weights of the model saved with EarlyStopping
    """
    if method == "import custom trained model":
        
        if decay_steps_mode=="on":
            
            with open(os.path.join(os.getcwd(), "{0}\\{1}_{2}dim_{3}batchsize_{4}lr_{5}decaymultiplier_{6}.json".format(folder_path_model_saved,
                                                                                                                        saved_model_name,
                                                                                                                        str(hparams[HP_EMBEDDING_DIM]), 
                                                                                                                        str(hparams[HP_HIDDEN_UNITS]), 
                                                                                                                        str(hparams[HP_LEARNING_RATE]), 
                                                                                                                        str(hparams[HP_DECAY_STEPS_MULTIPLIER]),
                                                                                                                        version_data_control)),'r') as f:
                model_json = json.load(f)

            model_imported = model_from_json(model_json)

            model_imported.load_weights(os.path.join(os.getcwd(), '{0}\\{1}_{2}dim_{3}batchsize_{4}lr_{5}decaymultiplier_{6}.h5'.format(folder_path_model_saved,
                                                                                                                                        saved_model_name,
                                                                                                                                        str(hparams[HP_EMBEDDING_DIM]), 
                                                                                                                                        str(hparams[HP_HIDDEN_UNITS]), 
                                                                                                                                        str(hparams[HP_LEARNING_RATE]), 
                                                                                                                                        str(hparams[HP_DECAY_STEPS_MULTIPLIER]),
                                                                                                                                        version_data_control)))
        else:
            with open(os.path.join(os.getcwd(), "{0}\\{1}_{2}dim_{3}batchsize_{4}lr_{5}.json".format(folder_path_model_saved,
                                                                                                     saved_model_name,
                                                                                                     str(hparams[HP_EMBEDDING_DIM]), 
                                                                                                     str(hparams[HP_HIDDEN_UNITS]), 
                                                                                                     str(hparams[HP_LEARNING_RATE]),
                                                                                                     version_data_control)),'r') as f:
                model_json = json.load(f)

            model_imported = model_from_json(model_json)

            model_imported.load_weights(os.path.join(os.getcwd(), '{0}\\{1}_{2}dim_{3}batchsize_{4}lr_{5}.h5'.format(folder_path_model_saved,
                                                                                                                     saved_model_name,
                                                                                                                     str(hparams[HP_EMBEDDING_DIM]), 
                                                                                                                     str(hparams[HP_HIDDEN_UNITS]), 
                                                                                                                     str(hparams[HP_LEARNING_RATE]),
                                                                                                                     version_data_control)))
        if optimizer_name=="adam":
            optimizer = optimizer_adam_v2(hparams)
        
        elif optimizer_name=="sgd":
            optimizer = optimizer_sgd_v1(hparams, "step decay")
            
        else:
            optimizer = optimizer_rmsprop_v1(hparams)
            
        model_imported.compile(optimizer=optimizer,
                               loss=neural_network_parameters['model_loss'],
                               metrics=neural_network_parameters['model_metric'])
        print(type(model_imported))
        print("\nModel is loaded successfully\n")
    
    else:
        
        with open(os.path.join(os.getcwd(), "{0}\\{1}_{2}dim_{3}batchsize_{4}lr_{5}decaymultiplier_{6}.json".format(folder_path_model_saved,
                                                                                                                    saved_model_name,
                                                                                                                    str(hparams[HP_EMBEDDING_DIM]), 
                                                                                                                    str(hparams[HP_HIDDEN_UNITS]), 
                                                                                                                    str(hparams[HP_LEARNING_RATE]), 
                                                                                                                    str(hparams[HP_DECAY_STEPS_MULTIPLIER]),
                                                                                                                    version_data_control)),'r') as f:
            model_json = json.load(f)

        model_imported = model_from_json(model_json, custom_objects={'KerasLayer':hub.KerasLayer})

        model_imported.load_weights(os.path.join(os.getcwd(), '{0}\\{1}_{2}dim_{3}batchsize_{4}lr_{5}decaymultiplier_{6}.h5'.format(folder_path_model_saved,
                                                                                                                                    saved_model_name,
                                                                                                                                    str(hparams[HP_EMBEDDING_DIM]), 
                                                                                                                                    str(hparams[HP_HIDDEN_UNITS]), 
                                                                                                                                    str(hparams[HP_LEARNING_RATE]), 
                                                                                                                                    str(hparams[HP_DECAY_STEPS_MULTIPLIER]),
                                                                                                                                    version_data_control)))

        optimizer = optimizer_adam_v2(hparams[HP_LEARNING_RATE], hparams[HP_DECAY_STEPS_MULTIPLIER], partial_x_train_actors_array.shape[0], optimizer_parameters['validation_split_ratio'], hparams[HP_HIDDEN_UNITS])

        model_imported.compile(optimizer=optimizer,
                               loss=neural_network_parameters['model_loss'],
                               metrics=neural_network_parameters['model_metric'])
        print(type(model_imported))
        print("\nModel is loaded successfully\n")
    
    return model_imported

#----------------------------------------------------------------------

# Function 2

def create_df_scoring_table(method, decay_steps_mode, model_tag, hparams, model):
    """
    Create a scoring dictionary to select the best out of the four models
    """
    if method == "import custom trained model":
        
        model_evaluation = model.evaluate([X_test_seq_actors, X_test_seq_plot, X_test_seq_features, X_test_seq_reviews, X_test_seq_title],
                                          y_test,
                                          batch_size=hparams[HP_HIDDEN_UNITS],
                                          verbose=2)

        y_test_pred_probs = model.predict([X_test_seq_actors, X_test_seq_plot, X_test_seq_features, X_test_seq_reviews, X_test_seq_title])
        y_test_predictions = (y_test_pred_probs>0.5).astype(int)

        variance = np.var(y_test_predictions)
        sse = np.mean((np.mean(y_test_predictions) - y_test)**2)
        bias = sse - variance

        hamming_loss_value = HammingLoss(mode='multilabel')
        hamming_loss_value.update_state(y_test, y_test_predictions)
        
        if decay_steps_mode=="on":
            
            df_scores=pd.DataFrame({'Tag Name':pd.Series(model_tag, dtype='str'),
                                    'Embedding Dimension tag':pd.Series(hparams[HP_EMBEDDING_DIM], dtype='int'),
                                    'Batch tag':pd.Series(hparams[HP_HIDDEN_UNITS], dtype='int'),
                                    'Learning Rate tag':pd.Series(hparams[HP_LEARNING_RATE], dtype='float'),
                                    'Decay Multiplier tag':pd.Series(hparams[HP_DECAY_STEPS_MULTIPLIER], dtype='int'),
                                    'Test Loss':pd.Series([model_evaluation[0]], dtype='float'),
                                    'Test Hamming Loss':pd.Series([model_evaluation[1]], dtype='float'),
                                    'Hamming Loss':pd.Series([hamming_loss_value.result().numpy()], dtype='float'),
                                    'Zero_one Loss':pd.Series([zero_one_loss(y_test, y_test_predictions, normalize=False)], dtype='float'),
                                    'F1_score':pd.Series([f1_score(y_test, y_test_predictions, average="micro")], dtype='float'),
                                    'F1_score_samples':pd.Series([f1_score(y_test, y_test_predictions, average="samples")], dtype='float'),
                                    'ROC_score':pd.Series([roc_auc_score(y_test, y_test_predictions, average="micro", multi_class="ovr")], dtype='float'),
                                    'ROC_score_samples':pd.Series([roc_auc_score(y_test, y_test_predictions, average="samples", multi_class="ovr")], dtype='float'),
                                    'Bias':pd.Series([bias], dtype='float'),
                                    'Variance':pd.Series([variance], dtype='float')
                                   })

            df_scores.to_pickle(os.path.join(os.getcwd(), "{0}\\{1}_{2}dim_{3}batchsize_{4}lr_{5}decaymultiplier_{6}.pkl".format(folder_path_model_saved,
                                                                                                                                 saved_df_scored_metric_name,
                                                                                                                                 str(hparams[HP_EMBEDDING_DIM]),
                                                                                                                                 str(hparams[HP_HIDDEN_UNITS]),
                                                                                                                                 str(hparams[HP_LEARNING_RATE]),
                                                                                                                                 str(hparams[HP_DECAY_STEPS_MULTIPLIER]),
                                                                                                                                 version_data_control)))
        else:
            
            df_scores=pd.DataFrame({'Tag Name':pd.Series(model_tag, dtype='str'),
                                    'Embedding Dimension tag':pd.Series(hparams[HP_EMBEDDING_DIM], dtype='int'),
                                    'Batch tag':pd.Series(hparams[HP_HIDDEN_UNITS], dtype='int'),
                                    'Learning Rate tag':pd.Series(hparams[HP_LEARNING_RATE], dtype='float'),
                                    'Test Loss':pd.Series([model_evaluation[0]], dtype='float'),
                                    'Test Hamming Loss':pd.Series([model_evaluation[1]], dtype='float'),
                                    'Hamming Loss':pd.Series([hamming_loss_value.result().numpy()], dtype='float'),
                                    'Zero_one Loss':pd.Series([zero_one_loss(y_test, y_test_predictions, normalize=False)], dtype='float'),
                                    'F1_score':pd.Series([f1_score(y_test, y_test_predictions, average="micro")], dtype='float'),
                                    'F1_score_samples':pd.Series([f1_score(y_test, y_test_predictions, average="samples")], dtype='float'),
                                    'ROC_score':pd.Series([roc_auc_score(y_test, y_test_predictions, average="micro", multi_class="ovr")], dtype='float'),
                                    'ROC_score_samples':pd.Series([roc_auc_score(y_test, y_test_predictions, average="samples", multi_class="ovr")], dtype='float'),
                                    'Bias':pd.Series([bias], dtype='float'),
                                    'Variance':pd.Series([variance], dtype='float')
                                   })

            df_scores.to_pickle(os.path.join(os.getcwd(), "{0}\\{1}_{2}dim_{3}batchsize_{4}lr_{5}.pkl".format(folder_path_model_saved,
                                                                                                              saved_df_scored_metric_name,
                                                                                                              str(hparams[HP_EMBEDDING_DIM]),
                                                                                                              str(hparams[HP_HIDDEN_UNITS]),
                                                                                                              str(hparams[HP_LEARNING_RATE]),
                                                                                                              version_data_control)))
    else:
        
        model_evaluation = model.evaluate([test_bytes_list_plot, test_bytes_list_features, test_bytes_list_reviews, test_bytes_list_title],
                                          test_label,
                                          batch_size=hparams[HP_HIDDEN_UNITS],
                                          verbose=2)

        y_test_pred_probs = model.predict([test_bytes_list_plot, test_bytes_list_features, test_bytes_list_reviews, test_bytes_list_title])
        y_test_predictions = (y_test_pred_probs>0.5).astype(int)

        variance = np.var(y_test_predictions)
        sse = np.mean((np.mean(y_test_predictions) - test_label)**2)
        bias = sse - variance

        hamming_loss_value = HammingLoss(mode='multilabel')
        hamming_loss_value.update_state(test_label, y_test_predictions)

        df_scores=pd.DataFrame({'Tag Name':pd.Series(model_tag, dtype='str'),
                                'Batch tag':pd.Series(hparams[HP_HIDDEN_UNITS], dtype='int'),
                                'Learning Rate tag':pd.Series(hparams[HP_LEARNING_RATE], dtype='float'),
                                'Decay Multiplier tag':pd.Series(hparams[HP_DECAY_STEPS_MULTIPLIER], dtype='int'),
                                'Test Loss':pd.Series([model_evaluation[0]], dtype='float'),
                                'Test Hamming Loss':pd.Series([model_evaluation[1]], dtype='float'),
                                'Hamming Loss':pd.Series([hamming_loss_value.result().numpy()], dtype='float'),
                                'Zero_one Loss':pd.Series([zero_one_loss(test_label, y_test_predictions, normalize=False)], dtype='float'),
                                'F1_score':pd.Series([f1_score(test_label, y_test_predictions, average="micro")], dtype='float'),
                                'F1_score_samples':pd.Series([f1_score(test_label, y_test_predictions, average="samples")], dtype='float'),
                                'ROC_score':pd.Series([roc_auc_score(test_label, y_test_predictions, average="micro", multi_class="ovr")], dtype='float'),
                                'ROC_score_samples':pd.Series([roc_auc_score(test_label, y_test_predictions, average="samples", multi_class="ovr")], dtype='float'),
                                'Bias':pd.Series([bias], dtype='float'),
                                'Variance':pd.Series([variance], dtype='float')
                               })

        df_scores.to_pickle(os.path.join(os.getcwd(), "{0}\\{1}_{2}dim_{3}batchsize_{4}lr_{5}decaymultiplier_{6}.pkl".format(folder_path_model_saved,
                                                                                                                             saved_df_scored_metric_name,
                                                                                                                             str(hparams[HP_EMBEDDING_DIM]), 
                                                                                                                             str(hparams[HP_HIDDEN_UNITS]), 
                                                                                                                             str(hparams[HP_LEARNING_RATE]), 
                                                                                                                             str(hparams[HP_DECAY_STEPS_MULTIPLIER]),
                                                                                                                             version_data_control)))
    return df_scores

**Load the models per Optimizer & Create a scoring dataframe for each model**

Step 1 of the selection plan (based on the written thesis documentation)

In [ ]:
model_method_creation="adam"

# First run the 36 models with batch 32, 64 and then the other 18 model of 128 batch size if your Ram is 16GB

In [ ]:
# The first 36 models (0-36)

list_models=[]
list_df=[]

if model_method_creation=="adam":

    HP_HIDDEN_UNITS = hp.HParam('batch_size', hp.Discrete([32, 64, 128]))
    HP_EMBEDDING_DIM = hp.HParam('embedding_dim', hp.Discrete([50, 100, 150]))
    HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([0.001, 0.01, 0.1]))
    HP_DECAY_STEPS_MULTIPLIER = hp.HParam('decay_steps_multiplier', hp.Discrete([10, 20]))

    for batch_size in HP_HIDDEN_UNITS.domain.values:
        for embedding_dim in HP_EMBEDDING_DIM.domain.values:
            for learning_rate in HP_LEARNING_RATE.domain.values:
                for decay_steps_multiplier in HP_DECAY_STEPS_MULTIPLIER.domain.values:
                    hparams = {
                        HP_HIDDEN_UNITS: batch_size,
                        HP_EMBEDDING_DIM: embedding_dim,
                        HP_LEARNING_RATE: learning_rate,
                        HP_DECAY_STEPS_MULTIPLIER: decay_steps_multiplier
                      }
                    print("{0}/{1}".format(len(list_models)+1, (len(HP_HIDDEN_UNITS.domain.values)*len(HP_EMBEDDING_DIM.domain.values)*len(HP_LEARNING_RATE.domain.values)*len(HP_DECAY_STEPS_MULTIPLIER.domain.values))))
                    model_object=import_trained_keras_model("import custom trained model", "on", model_method_creation, hparams)
                    df_object=create_df_scoring_table("import custom trained model", "on", "{0}-{1}".format(saved_model_name, len(list_models)+1), hparams, model_object)
                    list_models.append(model_object)
                    list_df.append(df_object)

else:
    HP_HIDDEN_UNITS = hp.HParam('batch_size', hp.Discrete([32, 64]))
    HP_EMBEDDING_DIM = hp.HParam('embedding_dim', hp.Discrete([50, 100, 150]))
    HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([0.001, 0.01, 0.1]))

    for batch_size in HP_HIDDEN_UNITS.domain.values:
        for embedding_dim in HP_EMBEDDING_DIM.domain.values:
            for learning_rate in HP_LEARNING_RATE.domain.values:
                hparams = {
                    HP_HIDDEN_UNITS: batch_size,
                    HP_EMBEDDING_DIM: embedding_dim,
                    HP_LEARNING_RATE: learning_rate
                  }
                print("{0}/{1}".format(len(list_models)+1, (len(HP_HIDDEN_UNITS.domain.values)*len(HP_EMBEDDING_DIM.domain.values)*len(HP_LEARNING_RATE.domain.values))))
                model_object=import_trained_keras_model("import custom trained model", "off", model_method_creation, hparams)
                df_object=create_df_scoring_table("import custom trained model", "off", "{0}-{1}".format(saved_model_name, len(list_models)+1), hparams, model_object)
                list_models.append(model_object)
                list_df.append(df_object)

#### Import the first 36 models trained by the Adam Optimizer - Keras custom neural network

Run this cell only if model_method_creation="adam"

In [ ]:
model_one, model_two, model_three, model_four, model_five, model_six, model_seven, model_eight, model_nine, model_ten, model_eleven, model_twelve=list_models[0:12]
model_thirteen, model_fourteen, model_fifteen, model_sixteen, model_seventeen, model_eighteen, model_nineteen, model_twenty, model_twenty_one, model_twenty_two, model_twenty_three, model_twenty_four=list_models[12:24]
model_twenty_five, model_twenty_six, model_twenty_seven, model_twenty_eight, model_twenty_nine, model_twenty_thirty, model_thirty_one, model_thirty_two, model_thirty_three, model_thirty_four, model_thirty_five, model_thirty_six=list_models[24:36]

df_scores_one, df_scores_two, df_scores_three, df_scores_four, df_scores_five, df_scores_six, df_scores_seven, df_scores_eight, df_scores_nine, df_scores_ten, df_scores_eleven, df_scores_twelve=list_df[0:12]
df_scores_thirteen, df_scores_fourteen, df_scores_fifteen, df_scores_sixteen, df_scores_seventeen, df_scores_eighteen, df_scores_nineteen, df_scores_twenty, df_scores_twenty_one, df_scores_twenty_two, df_scores_twenty_three, df_scores_twenty_four=list_df[12:24]
df_scores_twenty_five, df_scores_twenty_six, df_scores_twenty_seven, df_scores_twenty_eight, df_scores_twenty_nine, df_scores_thirty, df_scores_thirty_one, df_scores_thirty_two, df_scores_thirty_three, df_scores_thirty_four, df_scores_thirty_five, df_scores_thirty_six=list_df[24:36]

frames_adam_one=[df_scores_one, df_scores_two, df_scores_three, df_scores_four, df_scores_five, df_scores_six, df_scores_seven, df_scores_eight, df_scores_nine, df_scores_ten, df_scores_eleven, df_scores_twelve,
                 df_scores_thirteen, df_scores_fourteen, df_scores_fifteen, df_scores_sixteen, df_scores_seventeen, df_scores_eighteen, df_scores_nineteen, df_scores_twenty, df_scores_twenty_one, df_scores_twenty_two, df_scores_twenty_three, df_scores_twenty_four,
                 df_scores_twenty_five, df_scores_twenty_six, df_scores_twenty_seven, df_scores_twenty_eight, df_scores_twenty_nine, df_scores_thirty, df_scores_thirty_one, df_scores_thirty_two, df_scores_thirty_three, df_scores_thirty_four, df_scores_thirty_five, df_scores_thirty_six]

result_adam_one=pd.concat(frames_adam_one)
result_adam_one=result_adam_one.reset_index(drop=True)
result_adam_one.sort_values(by=['Hamming Loss', 'Zero_one Loss'])

In [ ]:
print(result_adam_one.to_latex(index=False))

In [ ]:
result_adam_one.to_pickle(os.path.join(os.getcwd(), "{0}\\{1}_{2}dim_{3}batchsize_{4}lr_{5}decaymultiplier_{6}.pkl".format(folder_path_model_saved,
                                                                                                                           "results_table_adam_one",
                                                                                                                           str(100), #Embedding size of the the best model estimator
                                                                                                                           str(32), #Batch size of the the best model estimator
                                                                                                                           str(0.001), #Learning rate of the the best model estimator
                                                                                                                           str(10),  #Decay Steps Multiplayer of the the best model estimator
                                                                                                                           version_data_control)))

Best model of 36 presented above is the model 7 with:
* Embedding size: 100
* Batch size: 32
* Learning rate: 0.001
* Decay Steps Multiplier: 10
* Hamming loss & Zeron-one loss: 0.003620 - 566.0

In [ ]:
# The rest 18 models (37-54)

list_models=[]
list_df=[]

if model_method_creation=="adam":

    HP_HIDDEN_UNITS = hp.HParam('batch_size', hp.Discrete([128]))
    HP_EMBEDDING_DIM = hp.HParam('embedding_dim', hp.Discrete([50, 100, 150]))
    HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([0.001, 0.01, 0.1]))
    HP_DECAY_STEPS_MULTIPLIER = hp.HParam('decay_steps_multiplier', hp.Discrete([10, 20]))

    for batch_size in HP_HIDDEN_UNITS.domain.values:
        for embedding_dim in HP_EMBEDDING_DIM.domain.values:
            for learning_rate in HP_LEARNING_RATE.domain.values:
                for decay_steps_multiplier in HP_DECAY_STEPS_MULTIPLIER.domain.values:
                    hparams = {
                        HP_HIDDEN_UNITS: batch_size,
                        HP_EMBEDDING_DIM: embedding_dim,
                        HP_LEARNING_RATE: learning_rate,
                        HP_DECAY_STEPS_MULTIPLIER: decay_steps_multiplier
                      }
                    print("{0}/{1}".format(len(list_models)+1, (len(HP_HIDDEN_UNITS.domain.values)*len(HP_EMBEDDING_DIM.domain.values)*len(HP_LEARNING_RATE.domain.values)*len(HP_DECAY_STEPS_MULTIPLIER.domain.values))))
                    model_object=import_trained_keras_model("import custom trained model", "on", model_method_creation, hparams)
                    df_object=create_df_scoring_table("import custom trained model", "on", "{0}-{1}".format(saved_model_name, len(list_models)+1), hparams, model_object)
                    list_models.append(model_object)
                    list_df.append(df_object)

else:
    HP_HIDDEN_UNITS = hp.HParam('batch_size', hp.Discrete([32, 64, 128]))
    HP_EMBEDDING_DIM = hp.HParam('embedding_dim', hp.Discrete([50, 100, 150]))
    HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([0.001, 0.01, 0.1]))

    for batch_size in HP_HIDDEN_UNITS.domain.values:
        for embedding_dim in HP_EMBEDDING_DIM.domain.values:
            for learning_rate in HP_LEARNING_RATE.domain.values:
                hparams = {
                    HP_HIDDEN_UNITS: batch_size,
                    HP_EMBEDDING_DIM: embedding_dim,
                    HP_LEARNING_RATE: learning_rate
                  }
                print("{0}/{1}".format(len(list_models)+1, (len(HP_HIDDEN_UNITS.domain.values)*len(HP_EMBEDDING_DIM.domain.values)*len(HP_LEARNING_RATE.domain.values))))
                model_object=import_trained_keras_model("import custom trained model", "off", model_method_creation, hparams)
                df_object=create_df_scoring_table("import custom trained model", "off", "{0}-{1}".format(saved_model_name, len(list_models)+1), hparams, model_object)
                list_models.append(model_object)
                list_df.append(df_object)

#### Import the rest 18 models trained by the Adam Optimizer - Keras custom neural network

Run this cell only if model_method_creation="adam"

In [ ]:
model_one, model_two, model_three, model_four, model_five, model_six, model_seven, model_eight, model_nine, model_ten, model_eleven, model_twelve=list_models[0:12]
model_thirteen, model_fourteen, model_fifteen, model_sixteen, model_seventeen, model_eighteen=list_models[12:18]

df_scores_one, df_scores_two, df_scores_three, df_scores_four, df_scores_five, df_scores_six, df_scores_seven, df_scores_eight, df_scores_nine, df_scores_ten, df_scores_eleven, df_scores_twelve=list_df[0:12]
df_scores_thirteen, df_scores_fourteen, df_scores_fifteen, df_scores_sixteen, df_scores_seventeen, df_scores_eighteen=list_df[12:18]

frames_adam_two=[df_scores_one, df_scores_two, df_scores_three, df_scores_four, df_scores_five, df_scores_six, df_scores_seven, df_scores_eight, df_scores_nine, df_scores_ten, df_scores_eleven, df_scores_twelve,
                 df_scores_thirteen, df_scores_fourteen, df_scores_fifteen, df_scores_sixteen, df_scores_seventeen, df_scores_eighteen]

result_adam_two=pd.concat(frames_adam_two)
result_adam_two=result_adam_two.reset_index(drop=True)
result_adam_two.sort_values(by=['Hamming Loss', 'Zero_one Loss'])

In [ ]:
print(result_adam_two.to_latex(index=False))

In [ ]:
result_adam_two.to_pickle(os.path.join(os.getcwd(), "{0}\\{1}_{2}dim_{3}batchsize_{4}lr_{5}decaymultiplier_{6}.pkl".format(folder_path_model_saved,
                                                                                                                           "results_table_adam_two",
                                                                                                                           str(100), #Embedding size of the the best model estimator
                                                                                                                           str(128), #Batch size of the the best model estimator
                                                                                                                           str(0.001), #Learning rate of the the best model estimator
                                                                                                                           str(20),  #Decay Steps Multiplayer of the the best model estimator
                                                                                                                           version_data_control)))

Best model of 36 (0-36) presented above is the model 7 with:
* Embedding size: 100
* Batch size: 32
* Learning rate: 0.001
* Decay Steps Multiplier: 10
* Hamming loss & Zeron-one loss: 0.003620 - 566.0
    
Best model of 18 (36-54) presented above is the model 44 with:
* Embedding size: 100
* Batch size: 128
* Learning rate: 0.001
* Decay Steps Multiplier: 20
* Hamming loss & Zeron-one loss: 0.003986 - 622.0

The best out of the two is the model 7

#### -----------------------------------------------------------------

#### Import all the models trained by the SGD Optimizer - Keras custom neural network

Run this cell only if model_method_creation="sgd"

In [ ]:
saved_model_name="multi_input_keras_model"
folder_path_model_saved="model_one\\sgd_models_20072020"
saved_df_scored_metric_name="df_metrics_multi_input_keras_model"

In [ ]:
model_method_creation="sgd"

In [ ]:
list_models=[]
list_df=[]

if model_method_creation=="adam":

    HP_HIDDEN_UNITS = hp.HParam('batch_size', hp.Discrete([32, 64, 128]))
    HP_EMBEDDING_DIM = hp.HParam('embedding_dim', hp.Discrete([50, 100, 150]))
    HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([0.001, 0.01, 0.1]))
    HP_DECAY_STEPS_MULTIPLIER = hp.HParam('decay_steps_multiplier', hp.Discrete([10, 20])) #only used in adam optimizer

    for batch_size in HP_HIDDEN_UNITS.domain.values:
        for embedding_dim in HP_EMBEDDING_DIM.domain.values:
            for learning_rate in HP_LEARNING_RATE.domain.values:
                for decay_steps_multiplier in HP_DECAY_STEPS_MULTIPLIER.domain.values:
                    hparams = {
                        HP_HIDDEN_UNITS: batch_size,
                        HP_EMBEDDING_DIM: embedding_dim,
                        HP_LEARNING_RATE: learning_rate,
                        HP_DECAY_STEPS_MULTIPLIER: decay_steps_multiplier
                      }
                    print("{0}/{1}".format(len(list_models)+1, (len(HP_HIDDEN_UNITS.domain.values)*len(HP_EMBEDDING_DIM.domain.values)*len(HP_LEARNING_RATE.domain.values)*len(HP_DECAY_STEPS_MULTIPLIER.domain.values))))
                    model_object=import_trained_keras_model("import custom trained model", "on", model_method_creation, hparams)
                    df_object=create_df_scoring_table("import custom trained model", "on", "{0}-{1}".format(saved_model_name, len(list_models)+1), hparams, model_object)
                    list_models.append(model_object)
                    list_df.append(df_object)

else:
    HP_HIDDEN_UNITS = hp.HParam('batch_size', hp.Discrete([32, 64, 128]))
    HP_EMBEDDING_DIM = hp.HParam('embedding_dim', hp.Discrete([50, 100, 150]))
    HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([0.001, 0.01, 0.1]))

    for batch_size in HP_HIDDEN_UNITS.domain.values:
        for embedding_dim in HP_EMBEDDING_DIM.domain.values:
            for learning_rate in HP_LEARNING_RATE.domain.values:
                hparams = {
                    HP_HIDDEN_UNITS: batch_size,
                    HP_EMBEDDING_DIM: embedding_dim,
                    HP_LEARNING_RATE: learning_rate
                  }
                print("{0}/{1}".format(len(list_models)+1, (len(HP_HIDDEN_UNITS.domain.values)*len(HP_EMBEDDING_DIM.domain.values)*len(HP_LEARNING_RATE.domain.values))))
                model_object=import_trained_keras_model("import custom trained model", "off", model_method_creation, hparams)
                df_object=create_df_scoring_table("import custom trained model", "off", "{0}-{1}".format(saved_model_name, len(list_models)+1), hparams, model_object)
                list_models.append(model_object)
                list_df.append(df_object)

In [ ]:
model_one, model_two, model_three, model_four, model_five, model_six, model_seven, model_eight, model_nine, model_ten, model_eleven, model_twelve=list_models[0:12]
model_thirteen, model_fourteen, model_fifteen, model_sixteen, model_seventeen, model_eighteen, model_nineteen, model_twenty, model_twenty_one, model_twenty_two, model_twenty_three, model_twenty_four=list_models[12:24]
model_twenty_five, model_twenty_six, model_twenty_seven=list_models[24:27]

df_scores_one, df_scores_two, df_scores_three, df_scores_four, df_scores_five, df_scores_six, df_scores_seven, df_scores_eight, df_scores_nine, df_scores_ten, df_scores_eleven, df_scores_twelve=list_df[0:12]
df_scores_thirteen, df_scores_fourteen, df_scores_fifteen, df_scores_sixteen, df_scores_seventeen, df_scores_eighteen, df_scores_nineteen, df_scores_twenty, df_scores_twenty_one, df_scores_twenty_two, df_scores_twenty_three, df_scores_twenty_four=list_df[12:24]
df_scores_twenty_five, df_scores_twenty_six, df_scores_twenty_seven=list_df[24:27]

frames_glove=[df_scores_one, df_scores_two, df_scores_three, df_scores_four, df_scores_five, df_scores_six, df_scores_seven, df_scores_eight, df_scores_nine, df_scores_ten, df_scores_eleven, df_scores_twelve,
              df_scores_thirteen, df_scores_fourteen, df_scores_fifteen, df_scores_sixteen, df_scores_seventeen, df_scores_eighteen, df_scores_nineteen, df_scores_twenty, df_scores_twenty_one, df_scores_twenty_two, df_scores_twenty_three, df_scores_twenty_four,
              df_scores_twenty_five, df_scores_twenty_six, df_scores_twenty_seven]

result_sgd=pd.concat(frames_glove)
result_sgd=result_sgd.reset_index(drop=True)
result_sgd.sort_values(by=['Hamming Loss', 'Zero_one Loss'])

In [ ]:
print(result_sgd.to_latex(index=False))

In [ ]:
result_sgd.to_pickle(os.path.join(os.getcwd(), "{0}\\{1}_{2}dim_{3}batchsize_{4}lr_{5}.pkl".format(folder_path_model_saved,
                                                                                                   "results_table_sgd",
                                                                                                   str(100), #Embedding size of the the best model estimator
                                                                                                   str(64), #Batch size of the the best model estimator
                                                                                                   str(0.1), #Learning rate of the the best model estimator
                                                                                                   version_data_control)))

Best model of 27 presented above is the model 15 with:
* Embedding size: 100
* Batch size: 64
* Learning rate: 0.1
* Hamming loss & Zeron-one loss: 0.009035 - 1363.0

#### -----------------------------------------------------------------

#### Import all the models trained by the RMSprop Optimizer - Keras custom neural network

Run this cell only if model_method_creation="rmsprop"

In [ ]:
# Initialize name variables

saved_model_name="multi_input_keras_model"
folder_path_model_saved="model_one\\rmsprop_models_20072020"
saved_df_scored_metric_name="df_metrics_multi_input_keras_model"

In [ ]:
model_method_creation="rmsprop"

In [ ]:
list_models=[]
list_df=[]

if model_method_creation=="adam":

    HP_HIDDEN_UNITS = hp.HParam('batch_size', hp.Discrete([32, 64, 128]))
    HP_EMBEDDING_DIM = hp.HParam('embedding_dim', hp.Discrete([50, 100, 150]))
    HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([0.001, 0.01, 0.1]))
    HP_DECAY_STEPS_MULTIPLIER = hp.HParam('decay_steps_multiplier', hp.Discrete([10])) #only used in adam optimizer

    for batch_size in HP_HIDDEN_UNITS.domain.values:
        for embedding_dim in HP_EMBEDDING_DIM.domain.values:
            for learning_rate in HP_LEARNING_RATE.domain.values:
                for decay_steps_multiplier in HP_DECAY_STEPS_MULTIPLIER.domain.values:
                    hparams = {
                        HP_HIDDEN_UNITS: batch_size,
                        HP_EMBEDDING_DIM: embedding_dim,
                        HP_LEARNING_RATE: learning_rate,
                        HP_DECAY_STEPS_MULTIPLIER: decay_steps_multiplier
                      }
                    print("{0}/{1}".format(len(list_models)+1, (len(HP_HIDDEN_UNITS.domain.values)*len(HP_EMBEDDING_DIM.domain.values)*len(HP_LEARNING_RATE.domain.values)*len(HP_DECAY_STEPS_MULTIPLIER.domain.values))))
                    model_object=import_trained_keras_model("import custom trained model", "on", model_method_creation, hparams)
                    df_object=create_df_scoring_table("import custom trained model", "on", "{0}-{1}".format(saved_model_name, len(list_models)+1), hparams, model_object)
                    list_models.append(model_object)
                    list_df.append(df_object)

else:
    HP_HIDDEN_UNITS = hp.HParam('batch_size', hp.Discrete([32, 64, 128]))
    HP_EMBEDDING_DIM = hp.HParam('embedding_dim', hp.Discrete([50, 100, 150]))
    HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([0.001, 0.01, 0.1]))

    for batch_size in HP_HIDDEN_UNITS.domain.values:
        for embedding_dim in HP_EMBEDDING_DIM.domain.values:
            for learning_rate in HP_LEARNING_RATE.domain.values:
                hparams = {
                    HP_HIDDEN_UNITS: batch_size,
                    HP_EMBEDDING_DIM: embedding_dim,
                    HP_LEARNING_RATE: learning_rate
                  }
                print("{0}/{1}".format(len(list_models)+1, (len(HP_HIDDEN_UNITS.domain.values)*len(HP_EMBEDDING_DIM.domain.values)*len(HP_LEARNING_RATE.domain.values))))
                model_object=import_trained_keras_model("import custom trained model", "off", model_method_creation, hparams)
                df_object=create_df_scoring_table("import custom trained model", "off", "{0}-{1}".format(saved_model_name, len(list_models)+1), hparams, model_object)
                list_models.append(model_object)
                list_df.append(df_object)

In [ ]:
model_one, model_two, model_three, model_four, model_five, model_six, model_seven, model_eight, model_nine, model_ten, model_eleven, model_twelve=list_models[0:12]
model_thirteen, model_fourteen, model_fifteen, model_sixteen, model_seventeen, model_eighteen, model_nineteen, model_twenty, model_twenty_one, model_twenty_two, model_twenty_three, model_twenty_four=list_models[12:24]
model_twenty_five, model_twenty_six, model_twenty_seven=list_models[24:27]

df_scores_one, df_scores_two, df_scores_three, df_scores_four, df_scores_five, df_scores_six, df_scores_seven, df_scores_eight, df_scores_nine, df_scores_ten, df_scores_eleven, df_scores_twelve=list_df[0:12]
df_scores_thirteen, df_scores_fourteen, df_scores_fifteen, df_scores_sixteen, df_scores_seventeen, df_scores_eighteen, df_scores_nineteen, df_scores_twenty, df_scores_twenty_one, df_scores_twenty_two, df_scores_twenty_three, df_scores_twenty_four=list_df[12:24]
df_scores_twenty_five, df_scores_twenty_six, df_scores_twenty_seven=list_df[24:27]

frames_rmsprop=[df_scores_one, df_scores_two, df_scores_three, df_scores_four, df_scores_five, df_scores_six, df_scores_seven, df_scores_eight, df_scores_nine, df_scores_ten, df_scores_eleven, df_scores_twelve,
                df_scores_thirteen, df_scores_fourteen, df_scores_fifteen, df_scores_sixteen, df_scores_seventeen, df_scores_eighteen, df_scores_nineteen, df_scores_twenty, df_scores_twenty_one, df_scores_twenty_two, df_scores_twenty_three, df_scores_twenty_four,
                df_scores_twenty_five, df_scores_twenty_six, df_scores_twenty_seven]

result_rmsprop=pd.concat(frames_rmsprop)
result_rmsprop=result_rmsprop.reset_index(drop=True)
result_rmsprop.sort_values(by=['Hamming Loss', 'Zero_one Loss'])

In [ ]:
print(result_rmsprop.to_latex(index=False))

In [ ]:
result_rmsprop.to_pickle(os.path.join(os.getcwd(), "{0}\\{1}_{2}dim_{3}batchsize_{4}lr_{5}.pkl".format(folder_path_model_saved,
                                                                                                       "results_table_rmsprop",
                                                                                                       str(150), #Embedding size of the the best model estimator
                                                                                                       str(64), #Batch size of the the best model estimator
                                                                                                       str(0.001), #Learning rate of the the best model estimator
                                                                                                       version_data_control)))

Best model of 27 presented above is the model 16 with:
* Embedding size: 150
* Batch size: 64
* Learning rate: 0.001
* Hamming loss & Zeron-one loss: 0.004064 - 637.0

# <b>- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -  </b>

The three final best models, one per optimizer, are:

**Adam**
* Model 7
* Embedding size: 100
* Batch size: 32
* Learning rate: 0.001
* Decay Steps Multiplier: 10
* Hamming loss & Zeron-one loss: 0.003620 - 566.0

**SGD**
* Model 15
* Embedding size: 100
* Batch size: 64
* Learning rate: 0.1
* Hamming loss & Zeron-one loss: 0.009035 - 1363.0

**RMSprop**
* Model 16
* Embedding size: 150
* Batch size: 64
* Learning rate: 0.001
* Hamming loss & Zeron-one loss: 0.004064 - 637.0

# <b>- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -  </b>

**Comparison 2: Create a classification report and a confusion matrix for the two closest models (per optimizer)** <br>
Additionally, create a bias-variance tradeoff tample per optimizer

In [ ]:
# Best model selected-Adam

HP_HIDDEN_UNITS = hp.HParam('batch_size', hp.Discrete([32]))
HP_EMBEDDING_DIM = hp.HParam('embedding_dim', hp.Discrete([100]))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([0.001]))
HP_DECAY_STEPS_MULTIPLIER = hp.HParam('decay_steps_multiplier', hp.Discrete([10]))

for batch_size in HP_HIDDEN_UNITS.domain.values:
    for embedding_dim in HP_EMBEDDING_DIM.domain.values:
        for learning_rate in HP_LEARNING_RATE.domain.values:
            for decay_steps_multiplier in HP_DECAY_STEPS_MULTIPLIER.domain.values:
                hparams_adam = {
                    HP_HIDDEN_UNITS: batch_size,
                    HP_EMBEDDING_DIM: embedding_dim,
                    HP_LEARNING_RATE: learning_rate,
                    HP_DECAY_STEPS_MULTIPLIER: decay_steps_multiplier
                  }
                folder_path_model_saved="model_one\\adam_v2_models_20072020"
                model_one = import_trained_keras_model("import custom trained model", "on", "adam", hparams_adam)

In [ ]:
# Best model selected-SGD

HP_HIDDEN_UNITS = hp.HParam('batch_size', hp.Discrete([64]))
HP_EMBEDDING_DIM = hp.HParam('embedding_dim', hp.Discrete([100]))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([0.1]))

for batch_size in HP_HIDDEN_UNITS.domain.values:
    for embedding_dim in HP_EMBEDDING_DIM.domain.values:
        for learning_rate in HP_LEARNING_RATE.domain.values:
            hparams_sgd = {
                HP_HIDDEN_UNITS: batch_size,
                HP_EMBEDDING_DIM: embedding_dim,
                HP_LEARNING_RATE: learning_rate
              }
            folder_path_model_saved="model_one\\sgd_models_20072020"
            model_two = import_trained_keras_model("import custom trained model", "off", "sgd", hparams_sgd)

In [ ]:
# Best model selected-RMSprop

HP_HIDDEN_UNITS = hp.HParam('batch_size', hp.Discrete([64]))
HP_EMBEDDING_DIM = hp.HParam('embedding_dim', hp.Discrete([150]))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([0.001]))

for batch_size in HP_HIDDEN_UNITS.domain.values:
    for embedding_dim in HP_EMBEDDING_DIM.domain.values:
        for learning_rate in HP_LEARNING_RATE.domain.values:
            hparams = {
                HP_HIDDEN_UNITS: batch_size,
                HP_EMBEDDING_DIM: embedding_dim,
                HP_LEARNING_RATE: learning_rate
              }
            folder_path_model_saved="model_one\\rmsprop_models_20072020"
            model_three = import_trained_keras_model("import custom trained model", "off", "rmsprop", hparams)

In [ ]:
saved_version_control="20072020"

history_dataframe_one=pd.read_pickle(os.path.join(os.getcwd(), "model_one\\adam_v2_models_{0}\\metrics_histogram_multi_input_keras_{1}dim_{2}batchsize_{3}lr_{4}decaymultiplier_16072020.pkl".format(saved_version_control, str(100), str(32), str(0.001), str(10))))
history_dataframe_two=pd.read_pickle(os.path.join(os.getcwd(), "model_one\\sgd_models_{0}\\metrics_histogram_multi_input_keras_{1}dim_{2}batchsize_{3}lr_16072020.pkl".format(saved_version_control, str(100), str(64), str(0.1))))
history_dataframe_three=pd.read_pickle(os.path.join(os.getcwd(), "model_one\\rmsprop_models_{0}\\metrics_histogram_multi_input_keras_{1}dim_{2}batchsize_{3}lr_16072020.pkl".format(saved_version_control, str(150), str(64), str(0.001))))

In [ ]:
def create_classification_table(model):
    
    y_test_pred_probs = model.predict([X_test_seq_actors, X_test_seq_plot, X_test_seq_features, X_test_seq_reviews, X_test_seq_title])
    y_test_predictions = (y_test_pred_probs>0.5).astype(int)
    
    variance = np.var(y_test_predictions)
    sse = np.mean((np.mean(y_test_predictions) - y_test)**2)
    bias = sse - variance

    classification_table = classification_report(y_true=y_test, y_pred=y_test_predictions)
    
    return classification_table, variance, bias

def create_confusion_matrix(mode, decay_steps_mode, embedding_dim_mode,  model, hparams):

    if mode == "custom trained model":
        
        if decay_steps_mode=="on":
            
            if embedding_dim_mode=="on":
        
                y_test_pred_probs = model.predict([X_test_seq_actors, X_test_seq_plot, X_test_seq_features, X_test_seq_reviews, X_test_seq_title])
                y_test_predictions = (y_test_pred_probs>0.5).astype(int)

                conf_mat=confusion_matrix(y_test.argmax(axis=1), y_test_predictions.argmax(axis=1))

                conf_matrix=pd.DataFrame(conf_mat,
                                         columns=genres_list,
                                         index=genres_list)

                conf_matrix.to_pickle(os.path.join(os.getcwd(), "{0}\\{1}_{2}dim_{3}batchsize_{4}lr_{5}decaymultiplier_{6}.pkl".format(folder_path_model_saved,
                                                                                                                                       "confusion_matrix",
                                                                                                                                       str(hparams[HP_EMBEDDING_DIM]), 
                                                                                                                                       str(hparams[HP_HIDDEN_UNITS]), 
                                                                                                                                       str(hparams[HP_LEARNING_RATE]), 
                                                                                                                                       str(hparams[HP_DECAY_STEPS_MULTIPLIER]),
                                                                                                                                       version_data_control)))
            else:
                
                y_test_pred_probs = model.predict([X_test_seq_actors, X_test_seq_plot, X_test_seq_features, X_test_seq_reviews, X_test_seq_title])
                y_test_predictions = (y_test_pred_probs>0.5).astype(int)

                conf_mat=confusion_matrix(y_test.argmax(axis=1), y_test_predictions.argmax(axis=1))

                conf_matrix=pd.DataFrame(conf_mat,
                                         columns=genres_list,
                                         index=genres_list)

                conf_matrix.to_pickle(os.path.join(os.getcwd(), "{0}\\{1}_{2}batchsize_{3}lr_{4}decaymultiplier_{5}.pkl".format(folder_path_model_saved,
                                                                                                                                "confusion_matrix",
                                                                                                                                str(hparams[HP_HIDDEN_UNITS]),
                                                                                                                                str(hparams[HP_LEARNING_RATE]),
                                                                                                                                str(hparams[HP_DECAY_STEPS_MULTIPLIER]),
                                                                                                                                version_data_control)))
        else:
            y_test_pred_probs = model.predict([X_test_seq_actors, X_test_seq_plot, X_test_seq_features, X_test_seq_reviews, X_test_seq_title])
            y_test_predictions = (y_test_pred_probs>0.5).astype(int)

            conf_mat=confusion_matrix(y_test.argmax(axis=1), y_test_predictions.argmax(axis=1))

            conf_matrix=pd.DataFrame(conf_mat,
                                     columns=genres_list,
                                     index=genres_list)

            conf_matrix.to_pickle(os.path.join(os.getcwd(), "{0}\\{1}_{2}dim_{3}batchsize_{4}lr_{5}.pkl".format(folder_path_model_saved,
                                                                                                                "confusion_matrix",
                                                                                                                str(hparams[HP_EMBEDDING_DIM]), 
                                                                                                                str(hparams[HP_HIDDEN_UNITS]),
                                                                                                                str(hparams[HP_LEARNING_RATE]),
                                                                                                                version_data_control)))
    else:
        
        y_test_pred_probs = model.predict([test_bytes_list_plot, test_bytes_list_features, test_bytes_list_reviews, test_bytes_list_title])
        y_test_predictions = (y_test_pred_probs>0.5).astype(int)

        conf_mat=confusion_matrix(test_label.argmax(axis=1), y_test_predictions.argmax(axis=1))

        conf_matrix=pd.DataFrame(conf_mat,
                                 columns=genres_list,
                                 index=genres_list)

        conf_matrix.to_pickle(os.path.join(os.getcwd(), "{0}\\{1}_{2}batchsize_{3}lr_{4}decaymultiplier_{5}.pkl".format(folder_path_model_saved,
                                                                                                                        "confusion_matrix",
                                                                                                                        str(hparams[HP_HIDDEN_UNITS]), 
                                                                                                                        str(hparams[HP_LEARNING_RATE]), 
                                                                                                                        str(hparams[HP_DECAY_STEPS_MULTIPLIER]),
                                                                                                                        version_data_control)))
    return conf_matrix

In [ ]:
#ADAM Optimizer

folder_path_model_saved="model_one\\adam_v2_models_20072020"

classification_table_one, variance_adam, bias_adam=create_classification_table(model_one)
print("Classification report for the best model estimator of the Adam optimizaion function:\n\n" + str(classification_table_one))

bias_variance_tradeoff_adam=pd.DataFrame({'Tag Name':pd.Series("model seven adam", dtype='str'),
                                          'Bias': pd.Series(bias_adam, dtype='str'),
                                          'Variance': pd.Series(variance_adam, dtype='str'), 
                                          'Average Training loss': pd.Series(np.mean(history_dataframe_one.loss), dtype='str'),
                                          'Average Validation loss': pd.Series(np.mean(history_dataframe_one.val_loss), dtype='str')})

In [ ]:
#### ---------------------------------------------------------------------------#ADAM Optimizer

HP_HIDDEN_UNITS = hp.HParam('batch_size', hp.Discrete([32]))
HP_EMBEDDING_DIM = hp.HParam('embedding_dim', hp.Discrete([100]))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([0.001]))
HP_DECAY_STEPS_MULTIPLIER = hp.HParam('decay_steps_multiplier', hp.Discrete([10]))

for batch_size in HP_HIDDEN_UNITS.domain.values:
    for embedding_dim in HP_EMBEDDING_DIM.domain.values:
        for learning_rate in HP_LEARNING_RATE.domain.values:
            for decay_steps_multiplier in HP_DECAY_STEPS_MULTIPLIER.domain.values:
                hparams_adam = {
                    HP_HIDDEN_UNITS: batch_size,
                    HP_EMBEDDING_DIM: embedding_dim,
                    HP_LEARNING_RATE: learning_rate,
                    HP_DECAY_STEPS_MULTIPLIER: decay_steps_multiplier
                  }
                folder_path_model_saved="model_one\\adam_v2_models_20072020"
                confusion_matrix_one=create_confusion_matrix("custom trained model", "on", "on", model_one, hparams_adam)
confusion_matrix_one

In [ ]:
print(confusion_matrix_one.to_latex(index=True))

#### ---------------------------------------------------------------------------

In [ ]:
#SGD Optimizer

folder_path_model_saved="model_one\\sgd_models_20072020"

classification_table_two, variance_sgd, bias_sgd=create_classification_table(model_two)
print("Classification report for the best model estimator of the SGD optimizaion function:\n\n" + str(classification_table_two))

bias_variance_tradeoff_sgd=pd.DataFrame({'Tag Name':pd.Series("model fifteen sgd", dtype='str'),
                                         'Bias': pd.Series(bias_sgd, dtype='str'),
                                         'Variance': pd.Series(variance_sgd, dtype='str'),
                                         'Average Training loss': pd.Series(np.mean(history_dataframe_two.loss), dtype='str'),
                                         'Average Validation loss': pd.Series(np.mean(history_dataframe_two.val_loss), dtype='str')})

In [ ]:
#SGD Optimizer

HP_HIDDEN_UNITS = hp.HParam('batch_size', hp.Discrete([64]))
HP_EMBEDDING_DIM = hp.HParam('embedding_dim', hp.Discrete([100]))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([0.1]))

for batch_size in HP_HIDDEN_UNITS.domain.values:
    for embedding_dim in HP_EMBEDDING_DIM.domain.values:
        for learning_rate in HP_LEARNING_RATE.domain.values:
            hparams_sgd = {
                HP_HIDDEN_UNITS: batch_size,
                HP_EMBEDDING_DIM: embedding_dim,
                HP_LEARNING_RATE: learning_rate
              }
            folder_path_model_saved="model_one\\sgd_models_20072020"
            confusion_matrix_two=create_confusion_matrix("custom trained model", "off", "on", model_two, hparams_sgd)
confusion_matrix_two

In [ ]:
print(confusion_matrix_two.to_latex(index=True))

#### ---------------------------------------------------------------------------

In [ ]:
#RMSprop Optimizer

folder_path_model_saved="model_one\\rmsprop_models_20072020"

classification_table_three, variance_rmsprop, bias_rmsprop=create_classification_table(model_three)
print("Classification report for the best model estimator of the SGD optimizaion function:\n\n" + str(classification_table_three))

bias_variance_tradeoff_rmsprop=pd.DataFrame({'Tag Name':pd.Series("model sixteen rmsprop", dtype='str'),
                                             'Bias': pd.Series(bias_rmsprop, dtype='str'),
                                             'Variance': pd.Series(variance_rmsprop, dtype='str'),
                                             'Average Training loss': pd.Series(np.mean(history_dataframe_three.loss), dtype='str'),
                                             'Average Validation loss': pd.Series(np.mean(history_dataframe_three.val_loss), dtype='str')})

In [ ]:
#RMSprop Optimizer

HP_HIDDEN_UNITS = hp.HParam('batch_size', hp.Discrete([64]))
HP_EMBEDDING_DIM = hp.HParam('embedding_dim', hp.Discrete([150]))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([0.001]))

for batch_size in HP_HIDDEN_UNITS.domain.values:
    for embedding_dim in HP_EMBEDDING_DIM.domain.values:
        for learning_rate in HP_LEARNING_RATE.domain.values:
            hparams_rmsprop = {
                HP_HIDDEN_UNITS: batch_size,
                HP_EMBEDDING_DIM: embedding_dim,
                HP_LEARNING_RATE: learning_rate
              }
            folder_path_model_saved="model_one\\rmsprop_models_20072020"
            confusion_matrix_three=create_confusion_matrix("custom trained model", "off", "on", model_three, hparams_rmsprop)
confusion_matrix_three

In [ ]:
print(confusion_matrix_three.to_latex(index=True))

In [ ]:
bias_variance_frames = [bias_variance_tradeoff_adam, bias_variance_tradeoff_sgd, bias_variance_tradeoff_rmsprop]
bias_variance_result = pd.concat(bias_variance_frames)
bias_variance_result = bias_variance_result.reset_index(drop=True)
bias_variance_result

In [ ]:
print(bias_variance_result.to_latex(index=True))

**Comparison 3: Test Accuracy - Test Score/Loss**

In [ ]:
# Concat the df_metric_score dataframes of the three models under review.

df_score_one=pd.read_pickle(os.path.join(os.getcwd(), "model_one\\adam_v2_models_20072020\\df_metrics_multi_input_keras_model_100dim_32batchsize_0.001lr_10decaymultiplier_16072020.pkl"))
df_score_two=pd.read_pickle(os.path.join(os.getcwd(), "model_one\\sgd_models_20072020\\df_metrics_multi_input_keras_model_100dim_64batchsize_0.1lr_16072020.pkl"))
df_score_three=pd.read_pickle(os.path.join(os.getcwd(), "model_one\\rmsprop_models_20072020\\df_metrics_multi_input_keras_model_150dim_64batchsize_0.001lr_16072020.pkl"))

result=pd.concat([df_score_one, df_score_two, df_score_three])
result=result.reset_index(drop=True)
result.head()

In [ ]:
print(result.to_latex(index=True))

In [ ]:
colormin = 'red'
colorother = 'black'
clrs_acc = [colormin if result['Test Hamming Loss'].iloc[row]== result['Test Hamming Loss'].min() else colorother for row in range(len(result['Test Hamming Loss']))]
clrs_loss = [colormin if result['Test Loss'].iloc[row]== result['Test Loss'].min() else colorother for row in range(len(result['Test Loss']))]

x=result['Tag Name'].values.tolist()
y=result['Test Hamming Loss'].values.tolist()
fig5 = go.Figure()
fig5.add_trace(go.Scatter(x=x, y=y,
                          mode='markers',
                          marker=dict(color=clrs_acc)
                        ))
fig5.update_layout(title="Hamming Loss on test set per model",
                   xaxis_title="Model number",
                   yaxis_title="Hamming Loss value/model")
fig5.show()

#--------------------------------------------------

fig6 = go.Figure()
fig6.add_trace(go.Scatter(x=result['Tag Name'].values.tolist(), 
                          y=result['Test Loss'].values.tolist(),
                          mode='markers',
                          marker=dict(color=clrs_loss)
                         ))

fig6.update_layout(title="Loss score on test set per model",
                  xaxis_title="Model number",
                  yaxis_title="Test loss/model")
fig6.show()

#### Comparison 4: Predicted vs Actual Genre Tags

In [ ]:
X_test=pd.read_pickle(os.path.join(os.getcwd(), "80-20 split_non-balanced\\text_tokenization_padded_sequences_13072020\\x_test_13072020.pkl"))

In [ ]:
def predict_genre_tags(indx, model, genres_list):
        
    test_sequence_actors = X_test_seq_actors[indx:indx+1]
    
    test_sequence_plot = X_test_seq_plot[indx:indx+1]
    
    test_sequence_features = X_test_seq_features[indx:indx+1]
    
    test_sequence_reviews = X_test_seq_reviews[indx:indx+1]
    
    test_sequence_title = X_test_seq_title[indx:indx+1]
    
    text_prediction = model.predict([test_sequence_actors, test_sequence_plot, test_sequence_features, test_sequence_reviews, test_sequence_title])
    
    [float(i) for i in text_prediction[0]]
    
    genres_length=len(X_test['reduced_genres'].iloc[indx])
    
    tag_probabilities = text_prediction[0][np.argsort(text_prediction[0])[-genres_length:]]
    
    indexes = np.argsort(text_prediction[0])[::-1][:genres_length]
    
    indexes = np.sort(indexes)
    
    predicted_tags = []
    
    predicted_tags = [genres_list[i] for i in indexes]
    
    return predicted_tags

def create_predictions_df(model, random_numbers_list, file_name, optimizer_name,  hparams):
    
    if optimizer_name=="adam":
    
        df_predictions = pd.DataFrame({'Movie Title': pd.Series([X_test['title'].iloc[random_numbers_list[0]]], dtype='str'),
                                       'Predicted Genre tags (top 3)': pd.Series([predict_genre_tags(random_numbers_list[0], model, genres_list)], dtype='str'),
                                       'Real Genre tags': pd.Series([X_test['reduced_genres'].iloc[random_numbers_list[0]]], dtype='str')})

        for i in range(len(random_numbers_list)):

            df_predictions = df_predictions.append({'Movie Title': X_test['title'].iloc[random_numbers_list[i]], 
                                                    'Predicted Genre tags (top 3)': predict_genre_tags(random_numbers_list[i], model, genres_list),
                                                    'Real Genre tags': X_test['reduced_genres'].iloc[random_numbers_list[i]]} , ignore_index=True)

        df_predictions = df_predictions.drop(df_predictions.index[0])
        df_predictions.to_pickle("model_one\\{0}\\{1}_df_predictions_{2}dim_{3}batchsize_{4}lr_{5}decatmultiplier_{6}.pkl".format(file_name, 
                                                                                                                                  optimizer_name, 
                                                                                                                                  str(hparams[HP_EMBEDDING_DIM]), 
                                                                                                                                  str(hparams[HP_HIDDEN_UNITS]), 
                                                                                                                                  str(hparams[HP_LEARNING_RATE]), 
                                                                                                                                  str(hparams[HP_DECAY_STEPS_MULTIPLIER]),
                                                                                                                                  version_data_control))
    else:
        
        df_predictions = pd.DataFrame({'Movie Title': pd.Series([X_test['title'].iloc[random_numbers_list[0]]], dtype='str'),
                                       'Predicted Genre tags (top 3)': pd.Series([predict_genre_tags(random_numbers_list[0], model, genres_list)], dtype='str'),
                                       'Real Genre tags': pd.Series([X_test['reduced_genres'].iloc[random_numbers_list[0]]], dtype='str')})

        for i in range(len(random_numbers_list)):

            df_predictions = df_predictions.append({'Movie Title': X_test['title'].iloc[random_numbers_list[i]], 
                                                    'Predicted Genre tags (top 3)': predict_genre_tags(random_numbers_list[i], model, genres_list),
                                                    'Real Genre tags': X_test['reduced_genres'].iloc[random_numbers_list[i]]} , ignore_index=True)

        df_predictions = df_predictions.drop(df_predictions.index[0])
        df_predictions.to_pickle("model_one\\{0}\\{1}_df_predictions_{2}dim_{3}batchsize_{4}lr_{5}.pkl".format(file_name, 
                                                                                                               optimizer_name, 
                                                                                                               str(hparams[HP_EMBEDDING_DIM]),
                                                                                                               str(hparams[HP_HIDDEN_UNITS]),
                                                                                                               str(hparams[HP_LEARNING_RATE]),
                                                                                                               version_data_control))
    return df_predictions

In [ ]:
random_numbers = random.sample(range(1, y_test.shape[0]), 20)

save_index_of_numbers = random_numbers

print("Randomly saved numbers to make predictions: {}".format(save_index_of_numbers))

In [ ]:
HP_HIDDEN_UNITS = hp.HParam('batch_size', hp.Discrete([32]))
HP_EMBEDDING_DIM = hp.HParam('embedding_dim', hp.Discrete([100]))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([0.001]))
HP_DECAY_STEPS_MULTIPLIER = hp.HParam('decay_steps_multiplier', hp.Discrete([10]))

for batch_size in HP_HIDDEN_UNITS.domain.values:
    for embedding_dim in HP_EMBEDDING_DIM.domain.values:
        for learning_rate in HP_LEARNING_RATE.domain.values:
            for decay_steps_multiplier in HP_DECAY_STEPS_MULTIPLIER.domain.values:
                hparams_adam = {
                    HP_HIDDEN_UNITS: batch_size,
                    HP_EMBEDDING_DIM: embedding_dim,
                    HP_LEARNING_RATE: learning_rate,
                    HP_DECAY_STEPS_MULTIPLIER: decay_steps_multiplier
                  }
                predictions_dataframe_one=create_predictions_df(model_one, save_index_of_numbers, "adam_v2_models_20072020", "adam", hparams_adam)
predictions_dataframe_one

In [ ]:
print(predictions_dataframe_one.to_latex(index=True))

In [ ]:
HP_HIDDEN_UNITS = hp.HParam('batch_size', hp.Discrete([64]))
HP_EMBEDDING_DIM = hp.HParam('embedding_dim', hp.Discrete([100]))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([0.1]))

for batch_size in HP_HIDDEN_UNITS.domain.values:
    for embedding_dim in HP_EMBEDDING_DIM.domain.values:
        for learning_rate in HP_LEARNING_RATE.domain.values:
            hparams_sgd = {
                HP_HIDDEN_UNITS: batch_size,
                HP_EMBEDDING_DIM: embedding_dim,
                HP_LEARNING_RATE: learning_rate
              }
            predictions_dataframe_two=create_predictions_df(model_two, save_index_of_numbers, "sgd_models_20072020", "sgd", hparams_sgd)
predictions_dataframe_two

In [ ]:
print(predictions_dataframe_two.to_latex(index=True))

In [ ]:
for batch_size in HP_HIDDEN_UNITS.domain.values:
    for embedding_dim in HP_EMBEDDING_DIM.domain.values:
        for learning_rate in HP_LEARNING_RATE.domain.values:
            hparams_rmsprop = {
                HP_HIDDEN_UNITS: batch_size,
                HP_EMBEDDING_DIM: embedding_dim,
                HP_LEARNING_RATE: learning_rate
              }
            predictions_dataframe_three=create_predictions_df(model_three, save_index_of_numbers, "rmsprop_models_20072020", "rmsprop", hparams_rmsprop)
predictions_dataframe_three

In [ ]:
print(predictions_dataframe_three.to_latex(index=True))

#### Comparison 5: Training and Validation plots

Before creating the learning curves for each of the three best model estimators, some examples of underfitting and overfitting learning curves are presents with random data.

In [ ]:
# Example 1-Underfitting Learning Curve part 1

fig1=go.Figure()

fig1.add_trace(go.Scatter(x=[0, 20, 40, 60, 80, 100], 
                          y=[0.005, 0.003, 0.001, 0.0009, 0.0007,  0.0005],
                          mode='lines+markers',
                          name="Train",
                          line=dict(color='rgb(102, 194, 165)')))

fig1.add_trace(go.Scatter(x=[0, 20, 40, 60, 80, 100], 
                          y=[0.05, 0.055, 0.065, 0.068, 0.07, 0.075],
                          mode='lines+markers',
                          name="Validation",
                          line=dict(color='rgb(252, 141, 98)')))

fig1.update_layout(template="simple_white",
                   title="Loss - Underfitting learning curve",
                   xaxis_title="Number of epochs",
                   yaxis_title="Loss/epoch")
fig1.show()

In [ ]:
# Example 2-Underfitting Learning Curve part 2

fig1=go.Figure()

fig1.add_trace(go.Scatter(x=[0, 20, 40, 60, 80, 100], 
                          y=[1.075, 1.071, 1.066, 1.061, 1.057, 1.037],
                          mode='lines+markers',
                          name="Train",
                          line=dict(color='rgb(102, 194, 165)')))

fig1.add_trace(go.Scatter(x=[0, 20, 40, 60, 80, 100], 
                          y=[1.065, 1.061, 1.056, 1.051, 1.047, 1.027],
                          mode='lines+markers',
                          name="Validation",
                          line=dict(color='rgb(252, 141, 98)')))

fig1.update_layout(template="simple_white",
                   title="Loss - Underfitting learning curve",
                   xaxis_title="Number of epochs",
                   yaxis_title="Loss/epoch")
fig1.show()

In [ ]:
# Example 3-Underfitting Learning Curve

fig1=go.Figure()

fig1.add_trace(go.Scatter(x=[0, 20, 40, 60, 80, 100], 
                          y=[1.075, 0.85, 0.45, 0.35, 0.25, 0.05],
                          mode='lines+markers',
                          name="Train",
                          line=dict(color='rgb(102, 194, 165)')))

fig1.add_trace(go.Scatter(x=[0, 20, 40, 60, 80, 100], 
                          y=[1.005, 0.75, 0.51, 0.28, 0.38, 0.49],
                          mode='lines+markers',
                          name="Validation",
                          line=dict(color='rgb(252, 141, 98)')))

fig1.update_layout(template="simple_white",
                   title="Loss - Overfitting learning curve",
                   xaxis_title="Number of epochs",
                   yaxis_title="Loss/epoch")
fig1.show()

In [ ]:
# Example 4-Good Fit Learning Curves

fig1=go.Figure()

fig1.add_trace(go.Scatter(x=[0, 20, 40, 60, 80, 100], 
                          y=[1.075, 0.85, 0.45, 0.35, 0.25, 0.05],
                          mode='lines+markers',
                          name="Train",
                          line=dict(color='rgb(102, 194, 165)')))

fig1.add_trace(go.Scatter(x=[0, 20, 40, 60, 80, 100], 
                          y=[1.069, 0.82, 0.43, 0.32, 0.22, 0.03],
                          mode='lines+markers',
                          name="Validation",
                          line=dict(color='rgb(252, 141, 98)')))

fig1.update_layout(template="simple_white",
                   title="Loss - Good fit learning curve",
                   xaxis_title="Number of epochs",
                   yaxis_title="Loss/epoch")
fig1.show()

Start of the pre-final step of the selection plan | Training-Validation Accuracy/Loss Learning Curves

**Hamming Loss performance models**

In [ ]:
colormin = 'black'
colormax = 'black'
colorother = 'rgb(252, 141, 98)'

clrs_acc_model_adam = [colormax if history_dataframe_one.val_hamming_loss.iloc[row]==history_dataframe_one.val_hamming_loss.min() else colorother for row in range(len(history_dataframe_one.val_hamming_loss))]
clrs_acc_model_sgd = [colormax if history_dataframe_two.val_hamming_loss.iloc[row]==history_dataframe_two.val_hamming_loss.min() else colorother for row in range(len(history_dataframe_two.val_hamming_loss))]
clrs_acc_model_rmsprop = [colormax if history_dataframe_three.val_hamming_loss.iloc[row]==history_dataframe_three.val_hamming_loss.min() else colorother for row in range(len(history_dataframe_three.val_hamming_loss))]

#Hamming Loss of Adam optimizer model
fig1=go.Figure()

fig1.add_trace(go.Scatter(x=history_dataframe_one.epoch.tolist(), 
                          y=history_dataframe_one.hamming_loss.tolist(),
                          mode='lines+markers',
                          name='Training Hamming Loss',
                          line=dict(color='rgb(102, 194, 165)')))

fig1.add_trace(go.Scatter(x=history_dataframe_one.epoch.tolist(), 
                          y=history_dataframe_one.val_hamming_loss.tolist(),
                          mode='lines+markers',
                          name='Validation Hamming Loss',
                          line=dict(color='rgb(252, 141, 98)'),
                          marker=dict(color=clrs_acc_model_adam)))

fig1.update_layout(template="simple_white",
                   title="Hamming Loss score on train & validation sets (Model estimator of the Adam Optimizer)",
                   xaxis_title="Number of epochs",
                   yaxis_title="Hamming Loss/epoch")

fig1.update_layout(showlegend=True,
                   annotations=[dict(x=history_dataframe_one.epoch[history_dataframe_one.val_hamming_loss==history_dataframe_one.val_hamming_loss.min()].tolist()[0],
                                     y=history_dataframe_one.val_hamming_loss.min(),
                                     xref="x",yref="y",
                                     text="Epoch with the highest validation Hamming Loss",
                                     showarrow=True,
                                     arrowhead=5,
                                     ax=0,ay=40)])

fig1.update_layout(legend_title_text='Training & Validation Hamming Loss points per epoch')

fig1.show()
#---------------------------------------------------------

#Hamming Loss of SGD optimizer model

fig2=go.Figure()

fig2.add_trace(go.Scatter(x=history_dataframe_two.epoch.tolist(), 
                          y=history_dataframe_two.hamming_loss.tolist(),
                          mode='lines+markers',
                          name='Training Hamming Loss',
                          line=dict(color='rgb(102, 194, 165)')))

fig2.add_trace(go.Scatter(x=history_dataframe_two.epoch.tolist(), 
                          y=history_dataframe_two.val_hamming_loss.tolist(),
                          mode='lines+markers',
                          name='Validation Hamming Loss',
                          line=dict(color='rgb(252, 141, 98)'),
                          marker=dict(color=clrs_acc_model_sgd)))

fig2.update_layout(template="simple_white",
                   title="Hamming Loss score on train & validation sets (Model estimator of the SGD Optimizer)",
                   xaxis_title="Number of epochs",
                   yaxis_title="Hamming Loss/epoch")

fig2.update_layout(showlegend=True,
                   annotations=[dict(x=history_dataframe_two.epoch[history_dataframe_two.val_hamming_loss==history_dataframe_two.val_hamming_loss.min()].tolist()[0],
                                     y=history_dataframe_two.val_hamming_loss.min(),
                                     xref="x",yref="y",
                                     text="Epoch with the highest validation Hamming Loss",
                                     showarrow=True,
                                     arrowhead=5,
                                     ax=0,ay=40)])

fig2.update_layout(legend_title_text='Training & Validation Hamming Loss points per epoch')

fig2.show()

#---------------------------------------------------------

#Hamming Loss of RMSprop optimizer model

fig3=go.Figure()

fig3.add_trace(go.Scatter(x=history_dataframe_three.epoch.tolist(),
                          y=history_dataframe_three.hamming_loss.tolist(),
                          mode='lines+markers',
                          name='Training Hamming Loss',
                          line=dict(color='rgb(102, 194, 165)')))

fig3.add_trace(go.Scatter(x=history_dataframe_three.epoch.tolist(), 
                          y=history_dataframe_three.val_hamming_loss.tolist(),
                          mode='lines+markers',
                          name='Validation Hamming Loss',
                          line=dict(color='rgb(252, 141, 98)'),
                          marker=dict(color=clrs_acc_model_rmsprop)))

fig3.update_layout(template="simple_white",
                   title="Hamming Loss score on train & validation sets (Model estimator of the RMSprop Optimizer)",
                   xaxis_title="Number of epochs",
                   yaxis_title="Hamming Loss/epoch")

fig3.update_layout(showlegend=True,
                   annotations=[dict(x=history_dataframe_three.epoch[history_dataframe_three.val_hamming_loss==history_dataframe_three.val_hamming_loss.min()].tolist()[0],
                                     y=history_dataframe_three.val_hamming_loss.min(),
                                     xref="x",yref="y",
                                     text="Epoch with the highest validation Hamming Loss",
                                     showarrow=True,
                                     arrowhead=5,
                                     ax=0,ay=40)])

fig3.update_layout(legend_title_text='Training & Validation Hamming Loss points per epoch')

fig3.show()

In [ ]:
colormin = 'black'
colorother = 'rgb(252, 141, 98)'

clrs_loss_model_adam=[colormin if history_dataframe_one.val_loss.iloc[row]==history_dataframe_one.val_loss.min() else colorother for row in range(len(history_dataframe_one.val_loss))]
clrs_loss_model_sgd=[colormin if history_dataframe_two.val_loss.iloc[row]==history_dataframe_two.val_loss.min() else colorother for row in range(len(history_dataframe_two.val_loss))]
clrs_loss_model_rmsprop=[colormin if history_dataframe_three.val_loss.iloc[row]==history_dataframe_three.val_loss.min() else colorother for row in range(len(history_dataframe_three.val_loss))]

#Loss of Adam optimizer model

fig3=go.Figure()

fig3.add_trace(go.Scatter(x=history_dataframe_one.epoch.tolist(), 
                          y=history_dataframe_one.loss.tolist(),
                          mode='lines+markers',
                          name='Training Loss',
                          line=dict(color='rgb(102, 194, 165)')))

fig3.add_trace(go.Scatter(x=history_dataframe_one.epoch.tolist(), 
                          y=history_dataframe_one.val_loss.tolist(),
                          mode='lines+markers',
                          name='Validation Loss',
                          line=dict(color='rgb(252, 141, 98)'),
                          marker=dict(color=clrs_loss_model_adam)))

fig3.update_layout(template="simple_white",
                   title="Loss score on train & validation sets (Model estimator of the Adam Optimizer)",
                   xaxis_title="Number of epochs",
                   yaxis_title="Loss/epoch")

fig3.update_layout(showlegend=True,
                   annotations=[dict(x=history_dataframe_one.epoch[history_dataframe_one.val_loss==history_dataframe_one.val_loss.min()].tolist()[0],
                                     y=history_dataframe_one.val_loss.min(),
                                     xref="x",yref="y",
                                     text="Epoch with the lowest validation loss",
                                     showarrow=True,
                                     arrowhead=5,
                                     ax=0,ay=-40)])
fig3.show()

#---------------------------------------------------------

#Loss of SGD optimizer model

fig4=go.Figure()

fig4.add_trace(go.Scatter(x=history_dataframe_two.epoch.tolist(), 
                          y=history_dataframe_two.loss.tolist(),
                          mode='lines+markers',
                          name='Training Loss',
                          line=dict(color='rgb(102, 194, 165)')))

fig4.add_trace(go.Scatter(x=history_dataframe_two.epoch.tolist(), 
                          y=history_dataframe_two.val_loss.tolist(),
                          mode='lines+markers',
                          name='Validation Loss',
                          line=dict(color='rgb(252, 141, 98)'),
                          marker=dict(color=clrs_loss_model_sgd)))

fig4.update_layout(template="simple_white",
                   title="Loss score on train & validation sets (Model estimator of the SGD Optimizer)",
                   xaxis_title="Number of epochs",
                   yaxis_title="Loss/epoch")

fig4.update_layout(showlegend=True,
                   annotations=[dict(x=history_dataframe_two.epoch[history_dataframe_two.val_loss==history_dataframe_two.val_loss.min()].tolist()[0],
                                     y=history_dataframe_two.val_loss.min(),
                                     xref="x",yref="y",
                                     text="Epoch with the lowest validation loss",
                                     showarrow=True,
                                     arrowhead=5,
                                     ax=0,ay=-65)])
fig4.show()

#---------------------------------------------------------

#Loss of RMSprop optimizer model

fig5=go.Figure()

fig5.add_trace(go.Scatter(x=history_dataframe_three.epoch.tolist(), 
                          y=history_dataframe_three.loss.tolist(),
                          mode='lines+markers',
                          name='Training Loss',
                          line=dict(color='rgb(102, 194, 165)')))

fig5.add_trace(go.Scatter(x=history_dataframe_three.epoch.tolist(), 
                          y=history_dataframe_three.val_loss.tolist(),
                          mode='lines+markers',
                          name='Validation Loss',
                          line=dict(color='rgb(252, 141, 98)'),
                          marker=dict(color=clrs_loss_model_rmsprop)))

fig5.update_layout(template="simple_white",
                   title="Loss score on train & validation sets (Model estimator of the RMSprop Optimizer)",
                   xaxis_title="Number of epochs",
                   yaxis_title="Loss/epoch")

fig5.update_layout(showlegend=True,
                   annotations=[dict(x=history_dataframe_three.epoch[history_dataframe_three.val_loss==history_dataframe_three.val_loss.min()].tolist()[0],
                                     y=history_dataframe_three.val_loss.min(),
                                     xref="x",yref="y",
                                     text="Epoch with the lowest validation loss",
                                     showarrow=True,
                                     arrowhead=5,
                                     ax=0,ay=-65)])
fig5.show()

In [ ]:
def visualize_model(model):
    return SVG(model_to_dot(model, show_shapes= True, show_layer_names=True, dpi=65,).create(prog='dot', format='svg'))
visualize_model(model_one)

In [ ]:
tf.keras.utils.plot_model(
model_one,
to_file="model.png",
show_shapes=True,
show_layer_names=True,
rankdir="TB",
expand_nested=False,
dpi=96,
)

<b>- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -  </b>

Model estimator trained on binary accuracy performance metric

In [ ]:
saved_version_data_control="22042020"

X_train_seq_actors=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\20000_max_features_cleaned_numbers\\x_train_seq_actors_80-20_non-balanced_20000_{0}.npy".format(saved_version_data_control)))
X_train_seq_plot=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\20000_max_features_cleaned_numbers\\x_train_seq_plot_80-20_non-balanced_20000_{0}.npy".format(saved_version_data_control)))
X_train_seq_features=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\20000_max_features_cleaned_numbers\\x_train_seq_features_80-20_non-balanced_20000_{0}.npy".format(saved_version_data_control)))
X_train_seq_reviews=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\20000_max_features_cleaned_numbers\\x_train_seq_reviews_80-20_non-balanced_20000_{0}.npy".format(saved_version_data_control)))

print("X_train data inputs have been loaded!\n")

X_test_seq_actors=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\20000_max_features_cleaned_numbers\\x_test_seq_actors_80-20_non-balanced_20000_{0}.npy".format(saved_version_data_control)))
X_test_seq_plot=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\20000_max_features_cleaned_numbers\\x_test_seq_plot_80-20_non-balanced_20000_{0}.npy".format(saved_version_data_control)))
X_test_seq_features=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\20000_max_features_cleaned_numbers\\x_test_seq_features_80-20_non-balanced_20000_{0}.npy".format(saved_version_data_control)))
X_test_seq_reviews=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\20000_max_features_cleaned_numbers\\x_test_seq_reviews_80-20_non-balanced_20000_{0}.npy".format(saved_version_data_control)))

print("X_test data inputs have been loaded!\n")

y_train=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\20000_max_features_cleaned_numbers\\y_train_80-20_non-balanced_20000_{0}.npy".format(saved_version_data_control)))
y_test=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\20000_max_features_cleaned_numbers\\y_test_80-20_non-balanced_20000_{0}.npy".format(saved_version_data_control)))

print("y_train & y_test have been loaded!\n")

In [ ]:
"""
Import the tokenizers of each input, fitted on part 3.1
"""
with open(os.path.join(os.getcwd(), '80-20 split_non-balanced\\20000_max_features_cleaned_numbers\\actors_tokenizer_20000_{0}.pkl'.format(saved_version_data_control)),'rb') as f:
    actors_tokenizer = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '80-20 split_non-balanced\\20000_max_features_cleaned_numbers\\plot_tokenizer_20000_{0}.pkl'.format(saved_version_data_control)),'rb') as f:
    plot_tokenizer = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '80-20 split_non-balanced\\20000_max_features_cleaned_numbers\\features_tokenizer_20000_{0}.pkl'.format(saved_version_data_control)),'rb') as f:
    features_tokenizer = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '80-20 split_non-balanced\\20000_max_features_cleaned_numbers\\reviews_tokenizer_20000_{0}.pkl'.format(saved_version_data_control)),'rb') as f:
    reviews_tokenizer = pickle.load(f)

try:
    assert len(actors_tokenizer.word_index)==20000
    assert len(plot_tokenizer.word_index)==20000
    assert len(features_tokenizer.word_index)==20000
    assert len(reviews_tokenizer.word_index)==20000
except AssertionError:
    print("ERROR: The vocabulary length for some of the tokenizers, is not equal to 20000. Please verify their lengths by running the following: len(actors_tokenizer.word_index)")

print("Tokenizers are loaded successfully!")

In [ ]:
"""
Import the X_train, X_test, y_train & y_test data pickled from dataset part 3.1
"""
saved_version_data_control="22042020"

X_train=pd.read_pickle(os.path.join(os.getcwd(), "pickled_data_per_part\\version_{0}\\X_train_all_inputs_{0}.pkl".format(saved_version_data_control)))
X_test=pd.read_pickle(os.path.join(os.getcwd(), "pickled_data_per_part\\version_{0}\\X_test_all_inputs_{0}.pkl".format(saved_version_data_control)))
y_train=pd.read_pickle(os.path.join(os.getcwd(), "pickled_data_per_part\\version_{0}\\y_train_all_inputs_{0}.pkl".format(saved_version_data_control)))
y_test=pd.read_pickle(os.path.join(os.getcwd(), "pickled_data_per_part\\version_{0}\\y_test_all_inputs_{0}.pkl".format(saved_version_data_control)))

assert X_train.shape[0]==y_train.shape[0]
assert X_test.shape[0]==y_test.shape[0]

In [ ]:
neural_network_parameters={}
optimizer_parameters={}

neural_network_parameters['model_loss'] = 'binary_crossentropy'
neural_network_parameters['model_metric'] = 'accuracy'
validation_split_ratio=0.8

def optimizer_adam_v2_accuracy(batch_size_value):

    optimizer_parameters['steps_per_epoch'] = int(np.ceil((X_train_seq_features.shape[0]*validation_split_ratio)//batch_size_value))
    optimizer_parameters['lr_schedule_learning_rate'] = 0.01
    optimizer_parameters['lr_schedule_decay_steps'] = optimizer_parameters['steps_per_epoch']*1000
    optimizer_parameters['lr_schedule_decay_rate'] = 1
    optimizer_parameters['staircase'] = False
    
    lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
        optimizer_parameters['lr_schedule_learning_rate'],
        decay_steps=optimizer_parameters['lr_schedule_decay_steps'],
        decay_rate=optimizer_parameters['lr_schedule_decay_rate'],
        staircase=optimizer_parameters['staircase'])
    
    return keras.optimizers.Adam(lr_schedule)

#### Model 16 - 300 embedding dimenstion & 128 batch size

In [ ]:
with open(os.path.join(os.getcwd(), 'model_one\\adam_v2_models_22042020\\multi_input_keras_model_{0}dim_{1}batchsize_{2}lr_{3}decaymultiplier_22042020.json'.format(str(300), str(128), str(0.01), str(1000))),'r') as f:
    model_json = json.load(f)

model_sixteen = model_from_json(model_json)

model_sixteen.load_weights(os.path.join(os.getcwd(), 'model_one\\adam_v2_models_22042020\\multi_input_keras_model_{0}dim_{1}batchsize_{2}lr_{3}decaymultiplier_22042020.h5'.format(str(300), str(128), str(0.01), str(1000))))

model_sixteen.compile(optimizer=optimizer_adam_v2_accuracy(128),
                      loss=neural_network_parameters['model_loss'],
                      metrics=[neural_network_parameters['model_metric']])

print(type(model_sixteen))
print("\nModel is loaded successfully")

In [ ]:
df_scores_adam_accuracy=pd.read_pickle(os.path.join(os.getcwd(), "model_one\\adam_v2_models_22042020\\df_metrics_multy_input_keras_300dim_128batchsize_26052020.pkl"))
df_scores_adam_accuracy['Bias'], df_scores_adam_accuracy['Variance']=[-0.00047,0.096426]
df_scores_adam_accuracy

In [ ]:
print(df_scores_adam_accuracy.to_latex(index=False))

In [ ]:
history_dataframe_one=pd.read_pickle(os.path.join(os.getcwd(), "model_one\\adam_v2_models_22042020\\metrics_histogram_multi_input_keras_300dim_128batchsize_22042020.pkl"))

In [ ]:
colormax = 'black'
colorother = 'rgb(252, 141, 98)'
clrs_acc_model = [colormax if history_dataframe_one.val_accuracy.iloc[row]==history_dataframe_one.val_accuracy.max() else colorother for row in range(len(history_dataframe_one.val_accuracy))]

clrs_loss=[colormin if history_dataframe_one.val_loss.iloc[row]==history_dataframe_one.val_loss.min() else colorother for row in range(len(history_dataframe_one.val_loss))]

#Accuracy of model four
fig11=go.Figure()

fig11.add_trace(go.Scatter(x=history_dataframe_one.epoch.tolist(), 
                          y=history_dataframe_one.accuracy.tolist(),
                          mode='lines+markers',
                          name='Training Accuracy',
                          line=dict(color='rgb(102, 194, 165)')))

fig11.add_trace(go.Scatter(x=history_dataframe_one.epoch.tolist(), 
                          y=history_dataframe_one.val_accuracy.tolist(),
                          mode='lines+markers',
                          name='Validation Accuracy',
                          line=dict(color='rgb(252, 141, 98)'),
                          marker=dict(color=clrs_acc_model)))

fig11.update_layout(template="simple_white",
                   title="Accuracy score on train & validation sets (adam model accuracy)",
                   xaxis_title="Number of epochs",
                   yaxis_title="Accuracy/epoch")

fig11.update_layout(showlegend=True,
                   annotations=[dict(x=history_dataframe_one.epoch[history_dataframe_one.val_accuracy==history_dataframe_one.val_accuracy.max()].tolist()[0],
                                     y=history_dataframe_one.val_accuracy.max(),
                                     xref="x",yref="y",
                                     text="Epoch with the highest validation accuracy",
                                     showarrow=True,
                                     arrowhead=5,
                                     ax=0,ay=40)])
fig11.show()

In [ ]:
colormin = 'black'
colorother = 'rgb(252, 141, 98)'

clrs_loss_model=[colormin if history_dataframe_one.val_loss.iloc[row]==history_dataframe_one.val_loss.min() else colorother for row in range(len(history_dataframe_one.val_loss))]

#Loss of model four

fig12=go.Figure()

fig12.add_trace(go.Scatter(x=history_dataframe_one.epoch.tolist(), 
                          y=history_dataframe_one.loss.tolist(),
                          mode='lines+markers',
                          name='Training Loss',
                          line=dict(color='rgb(102, 194, 165)')))

fig12.add_trace(go.Scatter(x=history_dataframe_one.epoch.tolist(), 
                          y=history_dataframe_one.val_loss.tolist(),
                          mode='lines+markers',
                          name='Validation Loss',
                          line=dict(color='rgb(252, 141, 98)'),
                          marker=dict(color=clrs_loss_model)))

fig12.update_layout(template="simple_white",
                   title="Loss score on train & validation sets (adam model trained on accuracy)",
                   xaxis_title="Number of epochs",
                   yaxis_title="Loss/epoch")

fig12.update_layout(showlegend=True,
                   annotations=[dict(x=history_dataframe_one.epoch[history_dataframe_one.val_loss==history_dataframe_one.val_loss.min()].tolist()[0],
                                     y=history_dataframe_one.val_loss.min(),
                                     xref="x",yref="y",
                                     text="Epoch with the lowest validation loss",
                                     showarrow=True,
                                     arrowhead=5,
                                     ax=0,ay=-40)])
fig12.show()